# Non-extensive introduction to Online Machine Learning


**Saulo Martiello Mastelini** (saulomastelini@gmail.com)

Contact:

- [Website](https://smastelini.github.io)
- [Github](https://github.com/smastelini)
- [Linkedin](https://www.linkedin.com/in/smastelini/)
- [ResearchGate](https://www.researchgate.net/profile/Saulo-Mastelini)

Copyright (c) 2022

---

**Disclaimer**

As the title implies, this material is not an extensive introduction to the topic. It is just my humble attempt to present a general overview of decades worth of research in an ever-expanding area.

Every process takes time. Therefore, a few minutes or hours are not enough to explore a whole research area. The idea is to find the balance between diving too deep into a topic and being too superficial. This hands-on talk is not formal, so feel free to interrupt me and ask questions anytime.

---

**If you want to explore further**

If you want to learn more about the topics discussed in this notebook, I suggest:

- [MOA book](https://moa.cms.waikato.ac.nz/book-html/): an open-access book that discusses a lot of themes related to data streams
- [River documentation](https://riverml.xyz/): it has plenty of examples, tutorials, and theoretical resources. It is constantly updated and expanded.

If you have a specific question that is not covered in the documentation, you can always open a new _Discussion_ on Github. For sure somebody will help you! To do that, you need to head to the [River](https://github.com/online-ml/river) repository and find the discussion tab.

Contributions are always welcome. River is open source and kept by a community. Even though you might not have a technical background, it is always possible to help. Fixing and expanding the documentation is just an example of possible ways to get involved. If you find a bug, please let us know! 😁

---

**About River**

River is an open-source project focused on online machine learning and stream mining. It is the result of a merger between two preceding open source projects:

- creme
- scikit-multiflow

creme and scikit-multiflow had a lot of overlap and also different strengths and weaknesses. After a long time of planning and discussing core design aspects, the maintainers of both projects joined forces and created River.

Hence, River has the best of both worlds and it is the result of years of learned lessons in the preceding tools. River is focused on both researchers and practicioneers. A lot of people help River keep growing, but the core development team is spread between France, New Zealand, Vietnam, and Brazil.

---

## Outline

1. Online learning? Why?
2. Batch vs. Online
3. Building blocks: some examples
4. Why dictionaries?
5. How to evaluate an online machine learning model?
    - `progressive_val_score`
    - label delay
6. Concept drift
7. Examples of algorithms
    1. Classification
        1. Hoeffding Tree
        2. Adaptive Random Forest
    2. Regression
        1. **Hoeffding Tree**
        2. **AMRules**
    3. Clustering
        1. k-Means

In [ ]:
# Necessary packages

# !pip install numpy
# !pip install scikit-learn

# Latest released version
# !pip install river

# Development version
#!pip install git+https://github.com/online-ml/river --upgrade

# 1. Online Learning? Why?

Q: Why should somebody care about updating models online? What about just training them once and using them?
A: Well, that is indeed enough for most cases.

Nonetheless, imagine that:

- The amount of data instances is huge
- It is not possible to store everything
- The available computational power is limited
    - CPUs
    - Memory
    - Battery
- Data is non-stationary and/or evolves through time

Q: Is it possible to use traditional machine learning in these cases?
A: Yes!

One can still use traditional or batch machine learning if:

- Data is stationary, i.e., a sufficiently large sample is enough to achieve generalization

or

- The speed at which data is produced or collected is not too high
    - In these cases, the batch-incremental approach is a possible solution

## 1.1 Batch-incremental

A batch machine learning model is retrained in this strategy at regular intervals. Hence, we must define a training window by following one among the possible approaches:

<img src="time_windows.png">

**Fonte:** Adapted from:

> Carnein, M. and Trautmann, H., 2019. Optimizing data stream representation: An extensive survey on stream clustering algorithms. Business & Information Systems Engineering, 61(3), pp.277-297.

- *Landmarks* are the most common choice for batch-incremental applications. The window length is the central concern.
    - The current model may become outdated if the window is too large
    - The model may fail to capture the underlying patterns in the data if the window is too small.
    - Concept drift is a serious problem
        - Drifts do not typically occur at predefined and regular intervals
    
**Attention**: batch-incremental != mini-batch.

Artificial neural networks can be trained incrementally or progressively, usually relying on mini-batches of data.

Challenges such as "catastrophic forgetting" are one of the main concerns tackled in the **continual learning** research field.

## 1.2. It is worth noting

Data streams are not necessarily, time series.

Q: What is the difference between data streams and time series?
A: Data streams do not necessarily have explicit temporal dependencies like time series. For instance, sensor networks.
    - Varying transmission speeds
    - Sensor failure
    - Network expansion
    - And so on...
    Hence, the arrival order does not matter... much, but it does

# 2. Batch vs. Online

The River website has a nice [tutorial](https://riverml.xyz/latest/examples/batch-to-online/) on going from batch to online ML. But let's give a general overview of the differences.

A typical batch ML evaluation pipeline might look like this:

In [ ]:
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

data = load_wine()

X, y = data.data, data.target
kf = KFold(shuffle=True, random_state=8, n_splits=10)

accs = []

for train, test in kf.split(X):
    X_tr, X_ts = X[train], X[test]
    y_tr, y_ts = y[train], y[test]
    
    dt  = DecisionTreeClassifier(max_depth=5, random_state=93)
    dt.fit(X_tr, y_tr)
    
    accs.append(accuracy_score(y_ts, dt.predict(X_ts)))

print(f"Mean accuracy: {sum(accs) / len(accs)}")

In [ ]:
len(X)

The dataset is loaded in the memory and entirely available for inspection. The decision tree algorithm is allowed to perform multiple passes over the (training) data. Validation data is never used for training.

In the end, we might take the complete dataset (training + validation) to build a "final model", given that we have already found a good set of hyperparameters. Once trained, this model will be used to predict the types of wine samples.

Let's see what an online ML evaluation might look like:

In [ ]:
from river import metrics
from river import stream
from river import tree


acc = metrics.Accuracy()
ht = tree.HoeffdingTreeClassifier(max_depth=5, grace_period=20)

for x, y in stream.iter_sklearn_dataset(load_wine()):
    # The evaluation metric is evaluated before the model actually learns from the instance
    acc.update(y, ht.predict_one(x))
    # The model is updated one instance at a time
    ht.learn_one(x, y)

print(f"Accuracy: {acc.get()}")

In [ ]:
x, y

We process the input data sequentially. Data might be loaded on demand from the disk, a web server, or anywhere.
Data does not need to fit into the available memory.

Each instance is first used for testing and then to update the learning model. Everything works in an instance-by-instance regimen.

If the underlying process is guaranteed to be stationary, we could shuffle the data before passing it to the model.

**Note:** we cannot directly compare both the obtained accuracy values, as the evaluation strategies are not the same.

# 3. Building blocks: some examples

In [ ]:
import numpy as np
from river import stats

After first glancing at the differences, let's take things slowly and reflect on the building blocks necessary to perform Online Machine Learning.

Let's suppose we want to keep statistics for continually arriving data. For instance, we want to calculate the mean and variance.


Time to simulate:

In [ ]:
import random

rng = random.Random(42)

In [ ]:
%%time

values = []
stds_batch = []

for _ in range(50000):
    v = rng.gauss(5, 3)
    values.append(v)

    stds_batch.append(np.std(values, ddof=1) if len(values) > 1 else 0)

In [ ]:
rng = random.Random(42)

In [ ]:
%%time

stds_incr = []
var = stats.Var(ddof=1)

for _ in range(50000):
    v = rng.gauss(5, 3)
    var.update(v)
    stds_incr.append(var.get() ** 0.5)

A lot faster! But does it work?

In [ ]:
s_errors = 0

for batch, incr in zip(stds_batch, stds_incr):
    s_errors += (batch - incr)

s_errors, s_errors / len(stds_batch)

I hope this is convincing! River's [stats](https://riverml.xyz/dev/api/overview/#stats) module has a lot of tools to calculate statistics 🧐

Many of these things are the building blocks of Online Machine Learning algorithms.

---

**Practical example: Variance using the Welford algorithm**

- We need some variables:
    - $n$: number of observations
    - $\overline{x}_n$: the sample mean, after $n$ observations
    - $M_{2, n}$: second-order statistic
- The variables are initialized as follows:
    - $\overline{x}_{0} \leftarrow 0$
    - $M_{2,0} \leftarrow 0$
- The variables are updated using the following expressions:
    - $\overline{x}_n = \overline{x}_{n-1} + \dfrac{x_n - \overline{x}_{n-1}}{n}$
    - $M_{2,n} = M_{2,n-1} + (x_n - \overline{x}_{n-1})(x_n - \overline{x}_n)$
- The sample variance is obtained using: $s_n^2 = \dfrac{M_{2,n}}{n-1}$, for every $n > 1$
- We also get a robust mean estimator for free! 🤓

---

# 4. Why dictionaries (or why using a sparse data representation)?

In River, we use dictionaries as the primary data type.

Dictionaries:

- Key x value: keys are unique
- Values accessed via keys instead of indices
- Sparse
- There is no explicit ordering
- Dynamic!
- Mixed data types

Examples:

In [ ]:
from datetime import datetime

x = {
    "potato": 3,
    "car": 2,
    "data": datetime.now(),
    "yes_or_no": "yes"
}

x

In [ ]:
x["one extra"] = True
x

In [ ]:
del x["data"]
x

**Tip**: dictionaries are very similar to JSON.

Let's compare dictionaries with the traditional approach, based on arrays:

In [ ]:
data = load_wine()

X, y = data.data, data.target

X[0, :], data.feature_names

In [ ]:
y[0], data.target_names

We are going to put sklearn to the test.

In [ ]:
X_tr, y_tr = X[:-2, :], y[:-2]
X_ts, y_ts = X[-2:, :], y[-2:]

X_tr.shape, X_ts.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_tr, y_tr)

nb.predict(X_ts)

What if one feature was missing?

In [ ]:
try:
    nb.predict(X_ts[:, 1:])
except ValueError as error:
    print(error)

That type of situation is not uncommon in online scenarios. New sensors appear, some fail, and so on. So we must be able to deal with this kind of situation.

The majority of the models in River can deal with missing and emerging features! 🎉

In [ ]:
from river import naive_bayes

gnb = naive_bayes.GaussianNB()
dataset = stream.iter_sklearn_dataset(load_wine())

rng = random.Random(42)

# Probability of ignoring a feature
del_chance = 0.2

n_incomplete = 0
for i, (x, y) in enumerate(dataset):
    if i == 176:
        break
    
    x_copy = x.copy()
    aux = 0
    for xi in x:
        if rng.random() <= del_chance:
            del x_copy[xi]
            aux = 1
        
        # Update the counter of incomplete instances
        n_incomplete += aux
    
    gnb.learn_one(x_copy, y)

In [ ]:
x, y

In [ ]:
gnb.predict_proba_one(x)

We are going to explicitly modify this last example:

In [ ]:
x, y = next(dataset)
list(x.keys())

Firstly, we make a copy and delete some features:

In [ ]:
x_copy = x.copy()

del x_copy["malic_acid"]
del x_copy["hue"]
del x_copy["flavanoids"]

x_copy

Will our model work?

In [ ]:
gnb.predict_proba_one(x_copy), y

What if new features appeared?

In [ ]:
x["1st extra"] = 7.89
x["2nd extra"] = 2

x

In [ ]:
gnb.learn_one(x, y)

gnb.predict_one({"1st extra": 7.8, "2nd extra": 1.5})

In [ ]:
np.unique(data.target, return_counts=True)

Each model implements different strategies to deal with missing or emerging features.

In our example, "1" was the majority class, and so was the prediction of GaussianNB. That is the best it can do since there is not enough information about the new features. But these new features are already part of the model and will be updated with more observations.

In [ ]:
gnb.gaussians

# 5. How to evaluate models?


In every example presented so far, when a new instance arrives, we first make a prediction and then use the new datum to update the model.
No cross-validation, leave-one-out, and so on.

This evaluation strategy is close to a real-world scenario: usually, we first get the inputs without labels, and predictions must be made. After some time, class labels arrive.
In our examples, the label is "revealed" after the model makes a prediction. A delay exists between predicting and getting the label in an even more realistic evaluation scenario. Sometimes, the label never arrives for some instances.

We call this type of evaluation strategy _progressive validation_ or _prequential_ evaluation.

I suggest checking this [blog post from Max Halford](https://maxhalford.github.io/blog/online-learning-evaluation/), for more details on that matter.

In River, we have a utility function `progressive_val_score` in the `evaluate` module that handles all the situations mentioned above.

In [ ]:
from river import evaluate
from river import metrics
from river.datasets import synth


def label_delay(x, y):
    return rng.randint(0, 100)


rng = random.Random(8)
dataset = synth.RandomRBF(seed_sample=7, seed_model=9)
model = tree.HoeffdingTreeClassifier()

# We can combine metrics using pipeline operators
metric = metrics.Accuracy() + metrics.MicroF1() + metrics.BalancedAccuracy()

evaluate.progressive_val_score(
    dataset=dataset.take(50000),
    model=model,
    metric=metric,
    print_every=5000,
    show_memory=True,
    show_time=True,
    delay=label_delay
)

In [ ]:
model.draw()

# 6. Concept drift

One of the main concerns in online machine learning is the fact that data distribution may not be stationary. What does that mean?

Let's first think about an example of stationary distribution:

> Big tech company X released a new neural network for the Y problem with 3 zillion parameters, trained for 6 months using enough energy to power up multiple cities. The training dataset had Z terabytes...

Well, the data does not change. Linguistic rules (in NLP) or visual semantics don't usually vary or evolve. Everything is static under the same data collection policy.

A dog will always be a dog. A word has a limited set of synonyms, and so on. The rule of the game does not change. But even in these scenarios, there are exceptions. What if the rules changed?

These changes or concept drift may occur in real-world problems. For example:

Consumer buying pattern (toilet paper, masks, and hand sanitizer at the beginning of Covid pandemics);
Renewable energy production: sunlight and wind are not predictable;
traffic and routes

An entire research field in online machine learning is devoted to creating concept drift detectors and learning algorithms capable of adapting to changes in the data distribution.

I am not an expert on this topic, but I will try to give you a simple example of how to apply a drift detector.

Let's suppose we have a classification problem and are monitoring our model's predictive performance. We denote by $0$ the cases where the model correctly classifies an instance and by $1$ the misclassifications.

In [ ]:
rng = random.Random(8)

for _ in range(10):
    print(rng.choices([0, 1], weights=[0.7, 0.3])[0])

We can feed these values to a drift detector:

In [ ]:
from river import drift

detector = drift.ADWIN(delta=0.01)

vals = rng.choices([0, 1], weights=[0.7, 0.3], k=500)
for i, v in enumerate(vals):
    detector.update(v)
    
    if detector.drift_detected:
        print(f"Drift detected: {i}")

What if the data distribution changes

In [ ]:
detector = drift.ADWIN(delta=0.05)

vals = rng.choices([0, 1], weights=[0.7, 0.3], k=500)
vals.extend(rng.choices([0, 1], weights=[0.2, 0.8], k=500))
for i, v in enumerate(vals):
    detector.update(v)
    
    if detector.drift_detected:
        print(f"Drift detected: {i}")

ADWIN is one of the most utilized drift detectors, but there many other algorithms. Non-supervised, semi-supervised, multivariate, and so on.
Usually, detectors are used as components of predictive models. Each models applies drift detectors in a different manner.

# 7. Algorithm examples

I will present some examples of classification, regression, and clustering algorithms for reference. The API access is always the same, so you can try your luck and check other examples in the documentation.


## 7.1. Classification

Algorithms projected for binary classification can be extended to the multiclass case by relying on the tools available in the `multiclass` module:

- `OneVsOneClassifier`
- `OneVsRestClassifier`
- `OutputCodeClassifier`

River also have basics tools to handle multi-output tasks. Any contributions are welcome!

### 7.1.1. Hoeffding Trees

One of the most popular families of online machine learning algorithms. They take this name because the statistical measure called Hoeffding bound is used to define when splits are performed. This heuristic ensures the decisions taken incrementally are similar to those performed by a batch decision tree algorithm.


There are three main variants of Hoeffding Trees:

- Hoeffding Tree: vanilla version
- Hoeffding Adaptive Tree: adds drift detectors to each decision node. If a drift is detected, a new subtree is trained in the background and eventually may replace the affected tree branch.
- Extremely Fast Decision Tree: quickly deploys splits but later revisits and improves its own decisions.

**Main hyperparameters:**

- `grace_period`: the interval between split attempts.
- `delta`: the split significance parameter. The split confidence `1 - delta`.
- `max_depth`: max height a tree might have.

I wrote a [tutorial](https://riverml.xyz/dev/user-guide/on-hoeffding-trees/) on Hoeffding Trees, where you can find more details about the algorithms.

**Example:**

In [ ]:
from river import tree


dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)
model = tree.HoeffdingAdaptiveTreeClassifier(seed=42)
metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=1000, show_memory=True, show_time=True)

We can visualize the tree structure:

In [ ]:
model.draw()

We can also inspect how decisions are made:

In [ ]:
dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)

x, y = next(dataset)

print(model.debug_one(x))

### 7.1.2. Adaptive Random Forest

Adaptive random forest (ARF) is an incremental version of Random Forests that combines the following ingredients:

- Randomized Hoeffding Trees as base learners
- Drifts detectors for each tree
    - New trees are trained in the background when drifts are detected
- Online bagging

ARFs have all the parameters of HTs and also some extra critical parameters:

- `warning_detector` and `drift_detector`
- `n_models`: the number of trees
- `max_features`: the maximum number of features considered during split attempts at each decision node

**Example:**

In [ ]:
from river import ensemble


dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)

model = ensemble.AdaptiveRandomForestClassifier(seed=8)
metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=1000, show_memory=True, show_time=True)

## 7.2. Regression


I will use the same dataset for every regression example:

In [ ]:
def get_friedman():
    return synth.Friedman(seed=101).take(20000)

In [ ]:
x, y = next(get_friedman())
x, y

### 7.2.2. Hoeffding Tree

(I research this topic)

We have three main types of HTs for regression tasks:

- `HoeffdingTreeRegressor`: vanilla regressor.
- `HoeffdingAdaptiveTreeRegressor`: the regression counterpart of the adaptive classification tree.
- `iSOUPTreeRegressor`: Hoeffding Tree for multi-target regression tasks

Besides the parameters presented in the classification version, other important parameters are:

- `leaf_prediction`: the prediction strategy (regression or model tree)
- `leaf_model`: the regression model used in model trees' leaves
- `splitter`: the decision split algorithm

In [ ]:
from river import preprocessing

# We can combine multiple metrics in our report
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | tree.HoeffdingTreeRegressor()

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

As usual, we can inspect how decisions are made:

In [ ]:
x, y = next(get_friedman())

print(model.debug_one(x))

In [ ]:
model[-1].draw()

### 7.2.3. AMRules

Adaptive Model Rules.

(I also research this topic)

Creates decision rules by relying on the Hoeffding Bound. AMRules also has anomaly detection capabilities to "skip" anomalous training samples.

It has parameters similar to those of HTs:

- `n_min`: equivalent to `grace_period`
- `pred_type`: equivalent to `leaf_prediction`
- `pred_model`: equivalent to `leaf_model`
- `splitter`

Other important parameters:

- `m_min`: minimum number of instances to observe before detecting anomalies.
- `drift_detector`: the drift detection algorithm used by each rule.
- `anomaly_threshold`: threshold to decide whether or not an instance is anomalous (the smaller the score value, the more anomalous the instance is).
- `ordered_rule_set`: defines whether only the first rule is used for detection (when set to `True`) or all the rules are used (`False`).

In [ ]:
from river import rules

metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | rules.AMRules(
    splitter=tree.splitter.TEBSTSplitter(digits=1),  #  <- this is part of my research
    drift_detector=drift.ADWIN(),
    ordered_rule_set=False,
    m_min=100,
    delta=0.01
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

We can also inspect the model:

In [ ]:
x, y = next(get_friedman())

print(model.debug_one(x))
print(f"True label: {y}")

In [ ]:
x_scaled = model["StandardScaler"].transform_one(x)

model["AMRules"].anomaly_score(x_scaled)

and the pipeline:

In [ ]:
model

## 7.3. Clustering

Incremental algorithms must adapt to changes in the data. For instance, new clusters might appear, some might disappear. I will show one example of algorithm:

### 7.3.1. k-Means

There are multiple incremental versions of k-Means out there. The version available in River adds a parameter called `halflife` which controls the the intensity of the incremental updates.


In [ ]:
from river import cluster

metric = metrics.Silhouette()
model = cluster.KMeans(seed=7)


for x, _ in get_friedman():
    metric.update(x, model.predict_one(x), model.centers)
    model.learn_one(x)

print(metric.get())

In [ ]:
metric = metrics.Silhouette()
model = cluster.KMeans(n_clusters=3, seed=7)


for x, _ in get_friedman():
    metric.update(x, model.predict_one(x), model.centers)
    model.learn_one(x)

print(metric.get())

And increase the `halflife` value.

In [ ]:
metric = metrics.Silhouette()
model = cluster.KMeans(n_clusters=3, seed=7, halflife=0.7)


for x, _ in get_friedman():
    metric.update(x, model.predict_one(x), model.centers)
    model.learn_one(x)

print(metric.get())

# Wrapping up

We can go much deeper into online machine learning solutions. There are multiple strategies to combine models, selecting the best model among a set thereof, and many other aspects. Online hyperparameter tuning is also an exciting research area.

I strongly suggest checking these additional resources to learn more about online machine learning:

**Tutorials:**

- [The art of using pipelines](https://riverml.xyz/latest/examples/the-art-of-using-pipelines/)
- [Working with imbalanced data](https://riverml.xyz/dev/examples/imbalanced-learning/)
- [Debbuging a pipeline](https://riverml.xyz/dev/examples/debugging-a-pipeline/)

**Resource hub:**

- [Awesome online machine learning](https://github.com/online-ml/awesome-online-machine-learning)


---

Thank you so much for having me!

Do you have any questions?